# Cross-validation
---

## Training/validation/test data sets

* **Training set**: el conjunto de datos para entrenar a sus modelos.
* **Validation set**: El conjunto de datos utilizado para probar el rendimiento de sus modelos que ha construido con los conjuntos de entrenamiento. En función de las prestaciones, se elige el mejor modelo (final).
* **Test set**: utilice este conjunto de datos para probar el rendimiento de su modelo final.

## K-folds cross validation steps (k=4 porejemplo).

* Paso 1: Divida sus datos en un conjunto de entrenamiento y un conjunto de pruebas (por ejemplo, 80% de entrenamiento y 20% de prueba). El equipo de prueba nunca se utilizará en la formación y selección de modelos. 
* paso 2: dividir el conjunto de entrenamiento en subconjuntos k (k=4) eqaul: 3 subconjuntos para el entrenamiento + 1 subconjunto para la validación.
* Paso 3: Entrenar sus modelos con los 3 subconjuntos y calcular una puntuación de rendimiento con el subconjunto restante.
* paso 4: elija un subconjunto diferente para la validación y luego repita el paso 3 hasta que cada subconjunto se haya utilizado como subconjunto de validación.
* paso 5: para una validación cruzada k=4 veces, cada modelo entrenado debe haber sido validado por 4 subconjuntos y por lo tanto tiene 4 puntajes de desempeño. Calcule el promedio de estas 4 puntuaciones de rendimiento para cada modelo. Utilice la puntuación media para seleccionar el mejor modelo final.
* paso 6: aplique su modelo final al **untouched** datos de prueba y ver cómo funciona.

## Eejemplo de k-folds cross validation

### Construir cuadrículas de parámetros

* cuadrícula de parámetros: una combinación de todos los parámetros variables de su modelo.
* ejemplo: Si quiero entrenar un modelo de regresión logística en 4 diferentes *regParam* y 3 diferentes *elasticNetParam*, tendré 3 x 4 = 12 modelos para entrenar y validar.
    

In [ ]:
from pyspark.ml.classification import LogisticRegression
blor = LogisticRegression(featuresCol='indexed_features', labelCol='label', family='binomial')

from pyspark.ml.tuning import ParamGridBuilder
param_grid = ParamGridBuilder().\
    addGrid(blor.regParam, [0, 0.5, 1, 2]).\
    addGrid(blor.elasticNetParam, [0, 0.5, 1]).\
    build()

### Divida los datos en grupos de entrenamiento y de prueba
* Consulte la página[logistic regression page](logistic-regression.ipynb) para ver qué datos utilizamos y cómo se generaron los conjuntos de entrenamiento y pruebas.

#### Run k (k=4) folds cross validation

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator()

from pyspark.ml.tuning import CrossValidator
cv = CrossValidator(estimator=blor, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=4)

cvModel = cv.fit(training)